In [1]:
suppressMessages(suppressWarnings({
    library(ggplot2)
    library(igraph)
    library(tidyverse)
    library(ggpubr)
}))

In [2]:
# load wilcox test results and plot barplot to show celltype non-speicfic dominate expression across species
species <- c("Hsap", "Mmus", "Pvit", "Pmar")
type <- c("WGD", "SSD")
mtx <- c("exp", "pct")

friedman_res <- Reduce(rbind, lapply(species, FUN = function(s){
    Reduce(rbind, lapply(type, FUN = function(t){
        Reduce(rbind, lapply(mtx, FUN = function(m){
            info <- read.delim(paste0(s,"/",m,"/",s, ".", m, ".", t, ".friedman_test.txt"), header = T)
            info$species <- s
            info$type <- t
            info$mtx <- m
            return(info)
        }))
    }))
}))

In [3]:
plot_res <- friedman_res %>% group_by(species, type, mtx) %>% summarise(
    total = n(),
    sign = sum(p < 0.01, na.rm = TRUE),
    insign = total - sign
    )

`summarise()` has grouped output by 'species', 'type'. You can override using
the `.groups` argument.


In [8]:
tmp <- plot_res %>% filter(mtx == 'exp') %>% group_by(species, type) %>% summarise(ratio = sign/total) %>% ungroup()
tmp

`summarise()` has grouped output by 'species'. You can override using the
`.groups` argument.


species,type,ratio
<chr>,<chr>,<dbl>
Hsap,SSD,0.7072310
Hsap,WGD,0.7687296
Mmus,SSD,0.6604178
Mmus,WGD,0.7097136
Pmar,SSD,0.7987599
Pmar,WGD,0.7117194
Pvit,SSD,0.7603993
Pvit,WGD,0.6966803


In [5]:
plot_res1 <- plot_res %>% pivot_longer(cols = c(sign, insign), 
               names_to = "variable", 
               values_to = "value")
plot_res1$variable <- factor(plot_res1$variable, levels = c("sign", "insign"))
plot_res1$type <- factor(plot_res1$type, levels = c("WGD", "SSD"))
plot_res1$species <- factor(plot_res1$species, levels = species)

In [6]:
pdf("summary_family.exp.significant_diff.Friedman.pdf", width = 5, height = 4)
plot_res1 %>% filter(mtx == "exp") %>% ggbarplot(x = "variable", y = "value", color = "variable", fill = "variable") + 
    scale_fill_manual(values=c("#999900","#666666"))+
    scale_color_manual(values=c("#999900","#666666")) +
    facet_grid(vars(type), vars(species)) 
dev.off()

pdf 
  2

In [7]:
pdf("summary_family.pct.significant_diff.Friedman.pdf", width = 5, height = 4)
plot_res1 %>% filter(mtx == "pct") %>% ggbarplot(x = "variable", y = "value", color = "variable", fill = "variable") + 
    scale_fill_manual(values=c("#999900","#666666"))+
    scale_color_manual(values=c("#999900","#666666")) +
    facet_grid(vars(type), vars(species)) 
dev.off()

pdf 
  2